In [1]:
import pandas as pd
import numpy as np
from scipy import interpolate as sci
from scipy.optimize import fsolve


def BondCfls(CpnRate, CpnPeriod, Maturity, FV=100, Amortized=False, PrincipalPaymentSchedule=np.empty(1),  
             TimeFromMaturity = np.empty(1)):
    """ 
    CpnRate: Επιτόκιο τοκομεριδίου
    CpnPeriod: Τοκοφόρος περίοδος (έτη)
    Maturity: Λήξη (έτη)
    fwd_t: Χρόνος παράδοσης προθεσμιακού συμβολαίου
    FV: Ονομαστική αξία
    Amortized: False/True - Χωρίς/με απόσβεση κεφαλαίου
    PrincipalPaymentSchedule: numpy array - ακολουθία χρεολυσιών (ροών αποπληρωμής κεφαλαίου)
    TimeFromMaturity: numpy array - ακολουθία χρόνων αποπληρωμής κεφαλαίου από τη λήξη
    Επιστρέφει:
    AccruedInterest: Δεδουλευμένοι τόκοι ομολόγου
    df_out: dataframe των επιμέρους ροών
    """
    CashflowTimes = np.arange(Maturity, 0, -CpnPeriod)
    CashflowTimes = np.flip(CashflowTimes)
    CashflowTimes = CashflowTimes[:, np.newaxis]
    if Amortized:
        CashflowTimesfromMaturity = CashflowTimes - Maturity
        t = np.hstack((CashflowTimesfromMaturity, CashflowTimes))
        Cfls_df = pd.DataFrame(t, columns=['CflTimesfromMaturity', 'CflTimes'])
        ppS = np.hstack((TimeFromMaturity[:,np.newaxis], PrincipalPaymentSchedule[:,np.newaxis]))
        PP_df = pd.DataFrame(ppS, columns=['CflTimesfromMaturity', 'PrincipalPayments'])
        Cfls_df = pd.merge(Cfls_df, PP_df, on='CflTimesfromMaturity', how='outer')
        Cfls_df = Cfls_df.fillna(0)
        Cfls_df['OutstandingPrincipal'] = FV - np.cumsum(Cfls_df['PrincipalPayments'])
        Cfls_df['OutstandingPrincipal'] = Cfls_df['OutstandingPrincipal'].shift(1) 
        Cfls_df.loc[0, 'OutstandingPrincipal'] = Cfls_df.loc[1, 'OutstandingPrincipal'] + Cfls_df.loc[0, 'PrincipalPayments'] 
        Cfls_df['Cpns'] = Cfls_df['OutstandingPrincipal']*CpnRate*CpnPeriod
        Cfls_df['TotalCfls'] = Cfls_df['Cpns'] + Cfls_df['PrincipalPayments']
        AccruedInterest = (CpnPeriod - Cfls_df.loc[0, 'CflTimes'])*CpnRate*Cfls_df.loc[0, 'OutstandingPrincipal']
        df_out = Cfls_df[['CflTimes', 'TotalCfls']]
    else:
        Cashflows = np.zeros_like(CashflowTimes) + CpnRate*CpnPeriod*FV; 
        Cashflows[-1] += FV
        AccruedInterest = (CpnPeriod - CashflowTimes[0])*CpnRate*FV
        AccruedInterest = AccruedInterest[0]
        t = np.hstack((CashflowTimes, Cashflows))
        df_out = pd.DataFrame(t, columns=['CflTimes', 'TotalCfls'])
    return AccruedInterest, df_out


def spot2VR(r, t, m):
    """ 
    t: numpy array - Λήξεις spot επιτοκίων (έτη)
    r: numpy array spot επιτοκίων
    m: object - συχνότητα ανατοκισμού της σποτκαμπύλης που εξάγεται 'C'/Integer
    Επιστρέφει:
    vr: numpy array spot παραγόντων αύξησης 
    df: numpy array spot παραγόντων προεξόφλησης
    """
    vr = 0*r
    cont_comp = m == 'C'
    int_comp = m!= 'C'
    vr1 = vr[int_comp]; r1 = r[int_comp]; t1 = t[int_comp]; m1 = m[int_comp];
    # m - compounding
    vr1 = (1 + r1/m1)**(m1*t1)
    # Simple compounding
    vr1[(t1 < 1/m1)] = (1 + r1[(t1 < 1/m1)] * t1[(t1 < 1/m1)]); 
    vr[cont_comp] = np.exp(r[cont_comp] * t[cont_comp]) 
    vr[int_comp] = vr1
    df = 1/vr
    return vr, df


def VR2spot(vr, t, m2):
    """ 
    t: numpy array - Λήξεις vr / spot επιτοκίων (έτη)
    vr: numpy array παραγόντων αύξησης 
    m2: object - συχνότητα ανατοκισμού της καμπύλης που εξάγεται 'C'/Integer
    Επιστρέφει:
    r2: numpy array spot επιτοκίων
    """
    if m2 == "C":
        r2 = np.log(vr)/t
    else:
        m2 = int(m2)
        r2 = ((vr)**(1/(t*m2))-1.0)*m2
        r2[t<1/m2] = ((vr[t<1/m2])-1.0)/t[t<1/m2]
    return r2


def spot2fwdcurve(t, rspot, compounding_spot, start_t, end_t, compounding_fwd):
    """ 
    t: numpy array - Λήξεις spot επιτοκίων (έτη)
    rspot: numpy array - Spot εποιτόκια (π.χ 0.05 για το 5%)
    compounding_spot: numpy array - συχνότητα ανατοκισμού του κάθε επιτοκίου.
    start_t: numpy array - αρχή προθεσμιακών καταθέσεων. Το t1 για r(t1,t2).
    end_t: numpy array - τέλος προθεσμιακών καταθέσεων. Το t2 για r(t1,t2).
    compounding_fwd: numpy array - συχνότητα ανατοκισμού για ΟΛΑ τα προθεσμιακά επιτόκια.
    Επιστρέφει:
    fwd_df: dataframe με ορίσματα και αποτελέσματα
    """
    vr, df =  spot2VR(rspot, t, compounding_spot)
    rspot_c = VR2spot(vr, t, 'C')
    f = sci.interp1d(t, rspot_c, fill_value="extrapolate")
    r1= f(start_t)
    r2 = f(end_t)
    # Τα αποθηκεύουμε όλα σε ένα dataframe
    fwd_df = pd.DataFrame(np.hstack((start_t, end_t, r1, r2)), columns = ["t1","t2", "r(0,t1)", "r(0,t2)"]); 
    # Υπολογίζουμε πρώτα τους παράγοντες αύξησης spot.
    fwd_df['vr(0,t1)'] = np.exp(start_t*r1);
    fwd_df['vr(0,t2)'] = np.exp(end_t*r2);
    # Ακολούθως, υπολογίζουμε τους forward παράγοντες αύξησης.
    fwd_df['vr(t1,t2)'] = fwd_df['vr(0,t2)']/fwd_df['vr(0,t1)']
    fwd_df['r(t1,t2)'] = VR2spot(fwd_df['vr(t1,t2)'], fwd_df['t2']-fwd_df['t1'], compounding_fwd)
    return fwd_df
    

def BondPriceOffSpotCurve(SpotRates, SpotTenors, SpotCompounding, CpnRate, CpnPeriod, Maturity, fwd_t=0, FV=100,
                          Amortized=False, PrincipalPaymentSchedule=np.empty(1),  TimeFromMaturity = np.empty(1)) :
    """ 
    SpotTenors: numpy array - Λήξεις spot επιτοκίων (έτη)
    SpotRates: numpy array - Spot εποιτόκια (π.χ 0.05 για το 5%)
    SpotCompounding: numpy array - συχνότητα ανατοκισμού του κάθε επιτοκίου
    CpnRate: Επιτόκιο τοκομεριδίου
    CpnPeriod: Τοκοφόρος περίοδος (έτη)
    Maturity: Λήξη (έτη)
    fwd_t: Χρόνος παράδοσης προθεσμιακού συμβολαίου
    FV: Ονομαστική αξία
    Amortized: False/True - Χωρίς/με απόσβεση κεφαλαίου
    PrincipalPaymentSchedule: numpy array - ακολουθία χρεολυσιών (ροών αποπληρωμής κεφαλαίου)
    TimeFromMaturity: numpy array - ακολουθία χρόνων αποπληρωμής κεφαλαίου από τη λήξη
    Επιστρέφει:
    Price_spot: Τρέχουσα (dirty) τιμή ομολόγου
    Price_spot: Προθεσμιακή (dirty) τιμή ομολόγου
    Cfls_df: dataframe με ροές, επιτόκια και spot/fwd dfs
    """
    AccInt, Cfls_df = BondCfls(CpnRate, CpnPeriod, Maturity, FV, Amortized, PrincipalPaymentSchedule, TimeFromMaturity)
    vr, df =  spot2VR(SpotRates, SpotTenors, SpotCompounding); 
    rspot_c = VR2spot(vr, SpotTenors, 'C'); 
    f = sci.interp1d(SpotTenors, rspot_c, fill_value="extrapolate")
    r_on_Cfls= f(Cfls_df.CflTimes)
    # Υπολογίζουμε πρώτα τους παράγοντες προεξόφλησης spot.
    Cfls_df['df(0,t)'] = np.exp(-Cfls_df.CflTimes*r_on_Cfls)
    r_on_t1= f(fwd_t)
    df_on_t1 = np.exp(-r_on_t1*fwd_t)
    # Και τους προθεσμιακούς παράγοντες προεξόφλησης - Προεξοφλούμε μόνο τις ροές που έπονται του fwd_t
    Cfls_df['df(t1,t)'] = np.where(Cfls_df['CflTimes'] > fwd_t, Cfls_df['df(0,t)']/df_on_t1, 0)
    Price_spot = Cfls_df['TotalCfls'].T @ Cfls_df['df(0,t)'] 
    Price_fwd = Cfls_df['TotalCfls'].T @ Cfls_df['df(t1,t)']
    return Price_spot, Price_fwd, Cfls_df

def BondPricebyYTM(YTM,CpnRate, CpnPeriod, Maturity,FV=100, Amortized=False, 
                          PrincipalPaymentSchedule=np.empty(1),  TimeFromMaturity = np.empty(1)):
    """
    YTM: Η απόδοση στη λήξη, προεξοφλητικό επιτόκιο με επίπεδη καμπύλη επιτοκίων
    CpnRate: Επιτόκιο τοκομεριδίου
    CpnPeriod: Τοκοφόρος περίοδος (έτη)
    Maturity: Λήξη (έτη)
    FV: Ονομαστική αξία
    Amortized: False/True - Χωρίς/με απόσβεση κεφαλαίου
    PrincipalPaymentSchedule: numpy array - ακολουθία χρεολυσιών (ροών αποπληρωμής κεφαλαίου)
    TimeFromMaturity: numpy array - ακολουθία χρόνων αποπληρωμής κεφαλαίου από τη λήξη
    Επιστρέφει: 
    Price_spot : Τρέχουσα (dirty) τιμή ομολόγου
    """
    CashflowTimes = np.arange(Maturity, 0, -CpnPeriod)
    SpotTenors = np.flip(CashflowTimes)
    SpotRates=np.zeros_like(SpotTenors)+YTM
    # Προσοχή: Το διάνυσμα πρέπει να είναι τύπου object για να μπορεί να λάβει τιμές χαρακτήρα.
    SpotCompounding = np.empty_like(SpotTenors, dtype=object); SpotCompounding[:] = 1/CpnPeriod
    Price_spot, Price_fwd, Cfls_df = BondPriceOffSpotCurve(SpotRates,SpotTenors,SpotCompounding, 
                                    CpnRate,CpnPeriod, Maturity, 0, FV, Amortized, PrincipalPaymentSchedule, TimeFromMaturity)
    return Price_spot

def YTM(MarketPrice, CpnRate, CpnPeriod, Maturity,FV=100, Amortized=False, 
                          PrincipalPaymentSchedule=np.empty(1),  TimeFromMaturity = np.empty(1)):
    """
    MarketPrice: Η τιμή του ομολόγου
    CpnRate: Επιτόκιο τοκομεριδίου
    CpnPeriod: Τοκοφόρος περίοδος (έτη)
    Maturity: Λήξη (έτη)
    FV: Ονομαστική αξία
    Amortized: False/True - Χωρίς/με απόσβεση κεφαλαίου
    PrincipalPaymentSchedule: numpy array - ακολουθία χρεολυσιών (ροών αποπληρωμής κεφαλαίου)
    TimeFromMaturity: numpy array - ακολουθία χρόνων αποπληρωμής κεφαλαίου από τη λήξη
    Επιστρέφει: 
    YTM_sol : Απόδοση στη λήξη
    """
    f= lambda x: BondPricebyYTM(x,CpnRate, CpnPeriod, Maturity,FV=100, Amortized=False, 
                          PrincipalPaymentSchedule=np.empty(1),  TimeFromMaturity = np.empty(1))-MarketPrice
    YTM_sol=fsolve(f,0.05)
    return YTM_sol